# 新段落

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
## Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import pandas as pd
import math

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
# torch.set_default_dtype(torch.float32)
torch.set_default_dtype(torch.float64)
torch.manual_seed(13)



# Sampling parameters etc
#n_axis = 51
n_axis = 41
#n_time = 131
n_time = 241

axis = torch.linspace(0,4,n_axis, device=device)
time = torch.linspace(0,8,n_time, device=device)
Ps = torch.cartesian_prod(axis,time)
# Number of points
lP = Ps.shape[0]


# Initial dataset
data_axis = torch.linspace(0,12, 1201, device=device)
data_time = torch.linspace(0,0,1, device=device)
data_Ps = torch.cartesian_prod(data_axis,time)

mask = data_Ps[:,1] == 0.
X = data_Ps[mask]
dX = data_Ps[mask]

mask1 = data_Ps[:,0] < 1e-10
Xbc = data_Ps[mask1]

Y = torch.exp(-(X[:,0]-3)**2*5)+torch.exp(-(X[:,0]-1)**2*10)+torch.exp(-(X[:,0]+3)**2*5)+torch.exp(-(X[:,0]+1)**2*10)
#Y = torch.cos((X[:,0]-1)*5)+torch.cos((X[:,1]-1)*5)
Y = Y.view(-1,1)

dY = -2*5*(X[:,0]-3)*torch.exp(-(X[:,0]-3)**2*5)+2*5*(X[:,0]+3)*torch.exp(-(X[:,0]+3)**2*5)
#dtY = -5*(torch.sin((X[:,0]-1)*5)+torch.sin((X[:,1]-1)*5))
dY = dY.view(-1,1)

Ybc = torch.where( ((Xbc[:,0]-1).abs() < 1e-1), 0., 0. )
Ybc = Ybc.view(-1,1)


dX = dX.to(torch.complex128)
dY = dY.to(torch.complex128)
#Xbc = Xbc.to(torch.complex128)
X = X.to(torch.complex128)
#Ybc = Ybc.to(torch.complex128)
Y = Y.to(torch.complex128)
#Y = torch.cat((Y,dY,Ybc),0)
Y = torch.cat((Y,dY),0)

Using cuda device


In [14]:
def getVarietyPoints(base1,base2):
    x1 = torch.cat(list(base1.unbind(1)), dim=0)
    x2 = torch.cat(list(base2.unbind(1)), dim=0)
    #x1 = base1.unbind(1)
    #x2 = base2.unbind(1)
    #return torch.stack([ torch.stack([x1,x1],1), torch.stack([x2,-x2],1) ])
    return torch.stack([ torch.stack([x1,x1],1), torch.stack([x2,-x2],1), torch.stack([-x1,x1],1), torch.stack([-x2,-x2],1) ])

def Phi(base1, base2, X):
    pts = getVarietyPoints(base1, base2)
    # return (pts.inner(X) * 1.j).exp().mean(0)
    return (pts.inner(X)).exp().mean(0)

def dPhi(base1, base2, X):
    pts = getVarietyPoints(base1, base2)
    return ((pts.inner(X)).exp().mul(pts[:,:,1].unsqueeze(2).repeat(1, 1, pts.inner(X).shape[2])).mean(0))

def dbcPhi(base1, base2, X):
    pts = getVarietyPoints(base1, base2)
    return ((pts.inner(X)).exp().mul(pts[:,:,0].unsqueeze(2).repeat(1, 1, pts.inner(X).shape[2])).mean(0))


def train(N):
    for epoch in range(N):
        PhiX = Phi(MC_base1 * 1.j, MC_base2 * 1.j, X)
        dPhiX = dPhi(MC_base1 * 1.j, MC_base2 * 1.j, dX)
        #dbcPhiX = dbcPhi(MC_base1 * 1.j, MC_base2 * 1.j, Xbc)
        #PhiX = torch.cat((PhiX,dPhiX,dbcPhiX),1)
        PhiX = torch.cat((PhiX,dPhiX),1)
        A = torch.diag_embed((eps - S_diag).exp()) + PhiX @ PhiX.H
        LA = torch.linalg.cholesky(A)
        alpha = torch.linalg.solve_triangular(LA, PhiX @ Y.to(torch.complex128), upper=False)

        nlml = 1/(2*eps.exp()) * (Y.norm().square() - alpha.norm().square())
        nlml += (PhiX.shape[1] - PhiX.shape[0])/2 * eps
        nlml += LA.diag().real.log().sum()
        nlml += 0.5*S_diag.sum()

        opt.zero_grad()
        nlml.backward()
        opt.step()

        with torch.no_grad():
            train_pred = PhiX.H @ torch.linalg.solve_triangular(LA.H, alpha, upper=True)
            err = (train_pred.real - Y).square().mean().sqrt()
            print(26*"~" + f'\nepoch {epoch}\n\
nlml {nlml}\n\
err {err}\n\
eps {eps.exp()}\n\
base1 std {MC_base1.std(0)}\n\
base2 std {MC_base2.std(0)}\n\
min,max {train_pred.real.min().detach(),train_pred.real.max().detach()}')

In [15]:
n_MC = 1000
# MC_axis = torch.linspace(-1,1, n_MC, device=device) * 30
MC_base1 = (torch.randn((n_MC, 1), device=device)).requires_grad_()
MC_base2 = (torch.randn((n_MC, 1), device=device)).requires_grad_()
# MC_base = torch.cartesian_prod(MC_axis,MC_axis).requires_grad_()
S_diag = torch.full((n_MC,), -np.log(n_MC), requires_grad=False, device=device)
# S_diag = torch.full((n_MC**2,), -np.log(n_MC**2), requires_grad=False, device=device)
eps = torch.tensor(np.log(1e-6), requires_grad=False, device=device)

In [16]:
opt = torch.optim.Adam([
    {'params': [MC_base1, MC_base2], 'lr': 1e-1},
    {'params': eps, 'lr': 1e-10}])
#train(100000)
train(20000)
opt = torch.optim.Adam([
    {'params': [MC_base1, MC_base2], 'lr': 1e-2},
    {'params': eps, 'lr': 1e-10}])
#train(100000)
train(20000)
opt = torch.optim.Adam([
    {'params': [MC_base1, MC_base2], 'lr': 1e-2},
    {'params': [S_diag], 'lr': 1e-2}])
train(1000)
opt = torch.optim.Adam([
    {'params': [MC_base1, MC_base2], 'lr': 1e-3},
    {'params': [S_diag], 'lr': 1e-3}])
train(300)
torch.save({
            'MC_base1': MC_base1.cpu(),
            'MC_base2': MC_base2.cpu(),
            'S_diag': S_diag.cpu(),
            'eps': eps.cpu(),
    }, "state.pt")


流式输出内容被截断，只能显示最后 5000 行内容。
~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch 675
nlml -15693.862891893594
err (5.554314231287026e-05-3.3763716893338202e-21j)
eps 1.0000000000000004e-06
base1 std tensor([4.3978], device='cuda:0')
base2 std tensor([4.3543], device='cuda:0')
min,max (tensor(-1.9178, device='cuda:0'), tensor(1.9178, device='cuda:0'))
~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch 676
nlml -15696.176276557413
err (5.2852156602152905e-05-3.1692746245052712e-21j)
eps 1.0000000000000004e-06
base1 std tensor([4.3979], device='cuda:0')
base2 std tensor([4.3543], device='cuda:0')
min,max (tensor(-1.9178, device='cuda:0'), tensor(1.9178, device='cuda:0'))
~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch 677
nlml -15695.693083132603
err (5.439463526780959e-05-3.3206805750505538e-21j)
eps 1.0000000000000004e-06
base1 std tensor([4.3979], device='cuda:0')
base2 std tensor([4.3543], device='cuda:0')
min,max (tensor(-1.9178, device='cuda:0'), tensor(1.9178, device='cuda:0'))
~~~~~~~~~~~~~~~~~~~~~~~~~~
epoch 678
nlml -15696.70643806

In [17]:
#torch.save({
#            'MC_base1': MC_base1.cpu(),
#            'MC_base2': MC_base2.cpu(),
#            'S_diag': S_diag.cpu(),
#            'eps': eps.cpu(),
#    }, "state.pt")


st = torch.load("state.pt")
MC_base1 = st['MC_base1']
MC_base2 = st['MC_base2']
S_diag = st['S_diag']
eps = st['eps']

# Prediction
#Phi_ = Phi(MC_base * 1.j, Ps.to(torch.complex128)).to(device)
Phi_ = Phi(MC_base1 * 1.j, MC_base2 * 1.j, Ps.to(torch.complex128).to("cpu"))
#dPhi_ = dPhi(MC_base1 * 1.j, MC_base2 * 1.j, Ps.to(torch.complex128).to("cpu"))
#dbcPhi_ = dbcPhi(MC_base1 * 1.j, MC_base2 * 1.j, Ps.to(torch.complex128).to("cpu"))
#Phi_ = torch.cat((Phi_,dPhi_,dbcPhi_),1)
PhiX = Phi(MC_base1 * 1.j, MC_base2 * 1.j, X.to("cpu"))
dPhiX = dPhi(MC_base1 * 1.j, MC_base2 * 1.j, dX.to("cpu"))
#dbcPhiX = dbcPhi(MC_base1 * 1.j, MC_base2 * 1.j, Xbc.to("cpu"))
#PhiX = torch.cat((PhiX,dPhiX,dbcPhiX),1)
PhiX = torch.cat((PhiX,dPhiX),1)
A = torch.diag_embed((eps - S_diag).exp()) + PhiX @ PhiX.H
LA = torch.linalg.cholesky(A)
alpha = torch.linalg.solve_triangular(LA, PhiX @ Y.to("cpu").to(torch.complex128), upper=False)
pred = Phi_.H @ torch.linalg.solve_triangular(LA.H, alpha.to("cpu"), upper=True)
pred = pred.real

pred.detach().cpu().numpy().tofile("pred.dat")
axis.cpu().numpy().tofile("axis.dat")
time.cpu().numpy().tofile("time.dat")

<ipython-input-17-d5b516231272>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  st = torch.load("state.pt")


In [18]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!cp pred.dat "/content/drive/MyDrive/Colab Notebooks"
!cp axis.dat "/content/drive/MyDrive/Colab Notebooks"
!cp time.dat "/content/drive/MyDrive/Colab Notebooks"
!cp state.pt "/content/drive/MyDrive/Colab Notebooks"

In [19]:
whitenoise=torch.randn(9881, 1)*0.0007

In [20]:
predC=pred+whitenoise.view_as(pred)

In [21]:
predC.detach().cpu().numpy().tofile("predC.dat")
!cp predC.dat "/content/drive/MyDrive/Colab Notebooks"